# Code Generator - Windows

Re-codes day4 excercise to build on windows

### Section 1: Manually Generate and Execute CPP Code

In [0]:
# imports

import os
import io
import sys
import json
import requests
from dotenv import load_dotenv
from openai import OpenAI
import google.generativeai
import anthropic
from IPython.display import Markdown, display, update_display
import gradio as gr
import subprocess
from huggingface_hub import login, InferenceClient
from transformers import AutoTokenizer
import platform

In [0]:
# environment

load_dotenv(override=True)
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY', 'your-key-if-not-using-env')
os.environ['ANTHROPIC_API_KEY'] = os.getenv('ANTHROPIC_API_KEY', 'your-key-if-not-using-env')
os.environ['HF_TOKEN'] = os.getenv('HF_TOKEN', 'your-key-if-not-using-env')

In [0]:
# initialize

openai = OpenAI()
claude = anthropic.Anthropic()
hf_token = os.environ['HF_TOKEN']
login(hf_token, add_to_git_credential=True)

# models
OPENAI_MODEL = "gpt-4o"
CLAUDE_MODEL = "claude-3-5-sonnet-20240620"
CODE_QWEN_MODEL = "Qwen/CodeQwen1.5-7B-Chat"
CODE_GEMMA_MODEL = "Gemini/CodeGemma-7B-IT"

# huggingface inference clients
# CODE_QWEN_URL = "https://h1vdol7jxhje3mpn.us-east-1.aws.endpoints.huggingface.cloud"
CODE_QWEN_URL = "https://mb4mgfmpql2yrady.us-east-1.aws.endpoints.huggingface.cloud"
CODE_GEMMA_URL = "https://c5hggiyqachmgnqg.us-east-1.aws.endpoints.huggingface.cloud"

# path to your visual studio build tools VsDevCmd.bat file - initialize environment
VISUAL_STUDIO_BUILD_TOOLS_PATH = "C:\\Program Files (x86)\\Microsoft Visual Studio\\2022\\BuildTools\\Common7\\Tools\\VsDevCmd.bat"

# prefix of the output files, e.g., "optimized_d4win.cpp", "optimized_d4win.exe", etc.
OUTPUT_FILE_NAME_BASE = "optimized_d4win"

In [0]:
# System message

system_message = "You are an assistant that reimplements Python code in high performance C++ for an M1 Mac. "
system_message += "Respond only with C++ code; use comments sparingly and do not provide any explanation other than occasional comments. "
system_message += "The C++ response needs to produce an identical output in the fastest possible time. Keep implementations of random number generators identical so that results match exactly."

In [0]:
# build user prompts and message structures for openai and claude models

def user_prompt_for(python):
    user_prompt = "Rewrite this Python code in C++ with the fastest possible implementation that produces identical output in the least time. "
    user_prompt += "Respond only with C++ code; do not explain your work other than a few comments. "
    user_prompt += "Pay attention to number types to ensure no int overflows. Remember to #include all necessary C++ packages such as iomanip.\n\n"
    user_prompt += python
    return user_prompt

def messages_for(python):
    return [
        {"role": "system", "content": system_message},
        {"role": "user", "content": user_prompt_for(python)}
    ]

In [0]:
# write to a file called <OUTPUT_FILE_NAME>.cpp

def write_output(cpp):
    code = cpp.replace("```cpp","").replace("```","")
    with open(f"{OUTPUT_FILE_NAME_BASE}.cpp", "w") as f:
        f.write(code)

In [0]:
# reade the contents of the output file called <OUTPUT_FILE_NAME>.cpp
def read_output():
    with open(f"{OUTPUT_FILE_NAME_BASE}.cpp", "r") as f:
        return f.read()

In [0]:
# optimize code using openai and claude

def optimize_gpt(python):    
    stream = openai.chat.completions.create(model=OPENAI_MODEL, messages=messages_for(python), stream=True)
    reply = ""
    for chunk in stream:
        fragment = chunk.choices[0].delta.content or ""
        reply += fragment
        print(fragment, end='', flush=True)
    write_output(reply)
    
def optimize_claude(python):
    result = claude.messages.stream(
        model=CLAUDE_MODEL,
        max_tokens=2000,
        system=system_message,
        messages=[{"role": "user", "content": user_prompt_for(python)}],
    )
    reply = ""
    with result as stream:
        for text in stream.text_stream:
            reply += text
            print(text, end="", flush=True)
    write_output(reply)

In [0]:
# generate build c++ comiple and run commands based on your platform
# Borrowed from @CloudLlama's contribution!

def run_cmd(command_to_run):
    try:
        run_result = subprocess.run(command_to_run, check=True, text=True, capture_output=True)
        return run_result.stdout if run_result.stdout else "SUCCESS"
    except:
        return ""
    
def c_compiler_cmd(filename_base):
    my_platform = platform.system()
    my_compiler = []
   
    try:                    
        if my_platform == "Windows":
            if os.path.isfile(VISUAL_STUDIO_BUILD_TOOLS_PATH):
                compile_cmd = ["cmd", "/c", VISUAL_STUDIO_BUILD_TOOLS_PATH, "&", "cl", f"{filename_base}.cpp"]
                my_compiler = ["Windows", "Visual Studio Build Tools", compile_cmd]
    
            if not my_compiler:
                my_compiler=[my_platform, "Unavailable", []]
                
        elif my_platform == "Linux":
            compile_cmd = ["g++", f"{filename_base}.cpp", "-o", filename_base]
            my_compiler = ["Linux", "GCC (g++)", compile_cmd]
    
            if not my_compiler:
                compile_cmd = ["clang++", f"{filename_base}.cpp", "-o", filename_base]
                my_compiler = ["Linux", "Clang++", compile_cmd]
        
            if not my_compiler:
                my_compiler=[my_platform, "Unavailable", []]
    
        elif my_platform == "Darwin":
            compile_cmd = ["clang++", "-Ofast", "-std=c++17", "-march=armv8.5-a", "-mtune=apple-m1", "-mcpu=apple-m1", "-o", filename_base, f"{filename_base}.cpp"]
            my_compiler = ["Macintosh", "Clang++", compile_cmd]
    
            if not my_compiler:
                my_compiler=[my_platform, "Unavailable", []]
    except:
        my_compiler=[my_platform, "Unavailable", []]
        
    if my_compiler:
        return my_compiler
    else:
        return ["Unknown", "Unavailable", []]    

In [0]:
# compile and execute the C++ code

def execute_cpp(code):
    filename_base = OUTPUT_FILE_NAME_BASE
    compiler_cmd = c_compiler_cmd(filename_base)
    write_output(code)
    
    try:
        if os.path.isfile(f"./{filename_base}.exe"):
            os.remove(f"./{filename_base}.exe")
        if os.path.isfile(f"./{filename_base}"):
            os.remove(f"./{filename_base}")
        compile_result = subprocess.run(compiler_cmd[2], check=True, text=True, capture_output=True)
        run_cmd = [f"./{filename_base}"]
        run_result = subprocess.run(run_cmd, check=True, text=True, capture_output=True)
        return run_result.stdout
    except subprocess.CalledProcessError as e:
        return f"An error occurred:\n{e.stderr}"

In [0]:
# simple pypton code to optimize

pi = """
import time

def calculate(iterations, param1, param2):
    result = 1.0
    for i in range(1, iterations+1):
        j = i * param1 - param2
        result -= (1/j)
        j = i * param1 + param2
        result += (1/j)
    return result

start_time = time.time()
result = calculate(100_000_000, 4, 1) * 4
end_time = time.time()

print(f"Result: {result:.12f}")
print(f"Execution Time: {(end_time - start_time):.6f} seconds")
"""

In [0]:
# execute the simple python code

exec(pi)

In [0]:
# optimize the simple python code with openai

optimize_gpt(pi)

In [0]:
# compile and run the openai optimized code

cpp_code = read_output()
execute_cpp(cpp_code)

In [0]:
# optimize the simple python code with claude

optimize_claude(pi)

In [0]:
# compile and run the claude optimized code

cpp_code = read_output()
execute_cpp(cpp_code)

In [0]:
# hard pypton code to optimize

pi_hard = """# Be careful to support large number sizes

def lcg(seed, a=1664525, c=1013904223, m=2**32):
    value = seed
    while True:
        value = (a * value + c) % m
        yield value
        
def max_subarray_sum(n, seed, min_val, max_val):
    lcg_gen = lcg(seed)
    random_numbers = [next(lcg_gen) % (max_val - min_val + 1) + min_val for _ in range(n)]
    max_sum = float('-inf')
    for i in range(n):
        current_sum = 0
        for j in range(i, n):
            current_sum += random_numbers[j]
            if current_sum > max_sum:
                max_sum = current_sum
    return max_sum

def total_max_subarray_sum(n, initial_seed, min_val, max_val):
    total_sum = 0
    lcg_gen = lcg(initial_seed)
    for _ in range(20):
        seed = next(lcg_gen)
        total_sum += max_subarray_sum(n, seed, min_val, max_val)
    return total_sum

# Parameters
n = 10000         # Number of random numbers
initial_seed = 42 # Initial seed for the LCG
min_val = -10     # Minimum value of random numbers
max_val = 10      # Maximum value of random numbers

# Timing the function
import time
start_time = time.time()
result = total_max_subarray_sum(n, initial_seed, min_val, max_val)
end_time = time.time()

print("Total Maximum Subarray Sum (20 runs):", result)
print("Execution Time: {:.6f} seconds".format(end_time - start_time))
"""

In [0]:
# execute the simple python code

exec(pi_hard)

In [0]:
# optimize the hard python code with openai

optimize_gpt(pi_hard)

In [0]:
# compile and run the openai optimized code

cpp_code = read_output()
execute_cpp(cpp_code)

In [0]:
# optimize the hard python code with claude

optimize_claude(pi_hard)

In [0]:
# compile and run the openai optimized code

cpp_code = read_output()
execute_cpp(cpp_code)

### Section 2: Using Gradio Interface to Generate and Execute CPP Code

In [0]:
# configure streaming responses from openai and claude

def stream_gpt(python):    
    stream = openai.chat.completions.create(model=OPENAI_MODEL, messages=messages_for(python), stream=True)
    reply = ""
    for chunk in stream:
        fragment = chunk.choices[0].delta.content or ""
        reply += fragment
        yield reply.replace('```cpp\n','').replace('```','')
        
def stream_claude(python):
    result = claude.messages.stream(
        model=CLAUDE_MODEL,
        max_tokens=2000,
        system=system_message,
        messages=[{"role": "user", "content": user_prompt_for(python)}],
    )
    reply = ""
    with result as stream:
        for text in stream.text_stream:
            reply += text
            yield reply.replace('```cpp\n','').replace('```','')

In [0]:
# optimize code based on selected model

def optimize(python, model):
    if model=="GPT":
        result = stream_gpt(python)
    elif model=="Claude":
        result = stream_claude(python)
    else:
        raise ValueError("Unknown model")
    for stream_so_far in result:
        yield stream_so_far     

In [0]:
# use gradio to optimize python code from the UI

with gr.Blocks() as ui:
    with gr.Row():
        python = gr.Textbox(label="Python code:", lines=10, value=pi)
        cpp = gr.Textbox(label="C++ code:", lines=10)
    with gr.Row():
        model = gr.Dropdown(["GPT", "Claude"], label="Select model", value="GPT")
        convert = gr.Button("Convert code")

    convert.click(optimize, inputs=[python, model], outputs=[cpp])

ui.launch(inbrowser=True)

In [0]:
# execute python code

def execute_python(code):
    try:
        output = io.StringIO()
        sys.stdout = output
        exec(code)
    finally:
        sys.stdout = sys.__stdout__
    return output.getvalue()

In [0]:
# css to color by code type

css = """
.python {background-color: #306998;}
.cpp {background-color: #050;}
"""

In [0]:
# add ability to run python code, compile and run c++ code in gradio

with gr.Blocks(css=css) as ui:
    gr.Markdown("## Convert code from Python to C++")
    with gr.Row():
        python = gr.Textbox(label="Python code:", value=pi, lines=10)
        cpp = gr.Textbox(label="C++ code:", lines=10)
    with gr.Row():
        model = gr.Dropdown(["GPT", "Claude"], label="Select model", value="GPT")
    with gr.Row():
        convert = gr.Button("Convert code")
    with gr.Row():
        python_run = gr.Button("Run Python")
        cpp_run = gr.Button("Run C++")
    with gr.Row():
        python_out = gr.TextArea(label="Python result:", elem_classes=["python"])
        cpp_out = gr.TextArea(label="C++ result:", elem_classes=["cpp"])

    convert.click(optimize, inputs=[python, model], outputs=[cpp])
    python_run.click(execute_python, inputs=[python], outputs=[python_out])
    cpp_run.click(execute_cpp, inputs=[cpp], outputs=[cpp_out])

ui.launch(inbrowser=True)

### Section 3: Add HuggingFace-hosted Open Source Models to Gradio App

In [0]:
# initilize code qwen huggingface endpoint

tokenizer = AutoTokenizer.from_pretrained(CODE_QWEN_MODEL)
messages = messages_for(pi)
text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

In [0]:
# inspect the message template (text)

print(text)

In [0]:
# manually call the huggingface endpoint, generate c++ with code qwen

client = InferenceClient(CODE_QWEN_URL, token=hf_token)
stream = client.text_generation(text, stream=True, details=True, max_new_tokens=3000)
for r in stream:
    print(r.token.text, end = "")

In [0]:
# configure streaming responses from code qwen

def stream_code_qwen(python):
    tokenizer = AutoTokenizer.from_pretrained(CODE_QWEN_MODEL)
    messages = messages_for(python)
    text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    client = InferenceClient(CODE_QWEN_URL, token=hf_token)
    stream = client.text_generation(text, stream=True, details=True, max_new_tokens=3000)
    result = ""
    for r in stream:
        result += r.token.text
        yield result   

In [0]:
# update optimize function to include code qwen

def optimize(python, model):
    if model=="GPT":
        result = stream_gpt(python)
    elif model=="Claude":
        result = stream_claude(python)
    elif model=="CodeQwen":
        result = stream_code_qwen(python)
    else:
        raise ValueError("Unknown model")
    for stream_so_far in result:
        yield stream_so_far    

In [0]:
# select the python code sample and return the code

def select_sample_program(sample_program):
    if sample_program=="pi":
        return pi
    elif sample_program=="pi_hard":
        return pi_hard
    else:
        return "Type your Python program here"

In [0]:
# run the updated gradio UI with code qwen

compiler_cmd = c_compiler_cmd("optimized")

with gr.Blocks(css=css) as ui:
    gr.Markdown("## Convert code from Python to C++")
    with gr.Row():
        python = gr.Textbox(label="Python code:", value=pi, lines=10)
        cpp = gr.Textbox(label="C++ code:", lines=10)
    with gr.Row():
        with gr.Column():
            sample_program = gr.Radio(["pi", "pi_hard"], label="Sample program", value="pi")
            model = gr.Dropdown(["GPT", "Claude", "CodeQwen"], label="Select model", value="GPT")
        with gr.Column():
            architecture = gr.Radio([compiler_cmd[0]], label="Architecture", interactive=False, value=compiler_cmd[0])
            compiler = gr.Radio([compiler_cmd[1]], label="Compiler", interactive=False, value=compiler_cmd[1])
    with gr.Row():
        convert = gr.Button("Convert code")
    with gr.Row():
        python_run = gr.Button("Run Python")
        if not compiler_cmd[1] == "Unavailable":
            cpp_run = gr.Button("Run C++")
        else:
            cpp_run = gr.Button("No compiler to run C++", interactive=False)
    with gr.Row():
        python_out = gr.TextArea(label="Python result:", elem_classes=["python"])
        cpp_out = gr.TextArea(label="C++ result:", elem_classes=["cpp"])

    sample_program.change(select_sample_program, inputs=[sample_program], outputs=[python])
    convert.click(optimize, inputs=[python, model], outputs=[cpp])
    python_run.click(execute_python, inputs=[python], outputs=[python_out])
    cpp_run.click(execute_cpp, inputs=[cpp], outputs=[cpp_out])

ui.launch(inbrowser=True)